#### Import Libraries

In [31]:
import openai
import pandas as pd
import numpy as np
import os
from openai.embeddings_utils import cosine_similarity
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import swifter
from configparser import ConfigParser

In [2]:
def join_path(a,b):
    '''
    Input
    a - folder location
    b - file name

    Return
    joining the path
    '''
    return os.path.join(a,b)

def read_data(path):

    try:
        return pd.read_excel(path)
    except:
        return pd.read_csv(path)


def messages_list(examples, input_levels,  choosen_level, input_row,sku_description_column_name):
    messages = []
    system_content = {"role" : "system", "content" : input_levels['system_content_part1'] + " " + input_levels['super_category'] + '.' +  f"In your response, mention ONLY the {input_levels[choosen_level]}."}
    messages.append(system_content)
    for i in range(examples.shape[0]):
        d = {}
        e = {}
        d = {"role" : "user", "content" : f"For this {input_levels['super_category']}, What is the {input_levels[choosen_level]}  of  the following SKU- {examples.user_response.values[i]}?"}
        messages.append(d)
        e = {"role" : "assistant", "content" : f"{examples[choosen_level].values[i]}"}
        messages.append(e)
    messages.append({"role" : "user", "content" : f"For this {input_levels['super_category']}, What is the {input_levels[choosen_level]} of {input_row[sku_description_column_name]}?"})
    return messages


def messages_list_conditional(examples, input_levels,  choosen_level, input_row, sku_description_column_name):
    messages = []
    system_content = {"role" : "system", "content" : input_levels['system_content_part1'] + " " + input_levels['super_category'] + '.' +  f"In your response, mention ONLY the {input_levels[choosen_level]}."}
    messages.append(system_content)
    for i in range(examples.shape[0]):
        d = {}
        e = {}
        d = {"role" : "user", "content" : f"Given the {input_levels['level1']} is {input_row['level1']}. What is the {input_levels[choosen_level]}  of  the following SKU- {examples.user_response.values[i]}?"}
        messages.append(d)
        e = {"role" : "assistant", "content" : f"{examples[choosen_level].values[i]}"}
        messages.append(e)
    messages.append({"role" : "user", "content" : f"What is the {input_levels[choosen_level]} of {input_row[sku_description_column_name]}?"})
    return messages

def retail_gpt(examples,input_levels, choosen_level, input_row, sku_description_column_name,  messages_list, model = 'gpt-3.5-turbo', max_tokens = 50, temperature = 0.6, ):
    response = openai.ChatCompletion.create(
    model = model,        
    messages = messages_list(examples=examples, input_levels=input_levels, choosen_level= choosen_level, input_row = input_row, sku_description_column_name = sku_description_column_name ),
    max_tokens = max_tokens,
    temperature = temperature
    )
    answer = response["choices"][0]['message']['content']
    return answer

def final_prediction(df, examples, input_levels, sku_description_column_name, messages_list,messages_list_conditional):
    df['level1_openai'] = df.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= 'level1',input_row= x,sku_description_column_name= sku_description_column_name,messages_list= messages_list), axis=1)

    # The for loop will work on levels level2 and onwards, conditional only on level 1
    for i in list(input_levels.keys())[-input_levels['number_of_levels'] + 1:]:
        df[i + '_openai'] = df.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= i,input_row= x,sku_description_column_name= sku_description_column_name, messages_list= messages_list_conditional), axis=1)

    return df

In [3]:
def final_prediction(df, examples, input_levels, sku_description_column_name, messages_list,messages_list_conditional):
    df['level1_openai'] = df.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= 'level1',input_row= x,sku_description_column_name= sku_description_column_name,messages_list= messages_list), axis=1)

    for i in list(input_levels.keys())[-input_levels['number_of_levels'] + 1:]:
        df[i + '_openai'] = df.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= i,input_row= x,sku_description_column_name= sku_description_column_name, messages_list= messages_list_conditional), axis=1)

    return df

#### File Path

In [3]:
import os
DIR_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
SRC_DIR = os.path.join(DIR_PATH, "src")
DATA_DIR = os.path.join(DIR_PATH, "data")
RAW_DIR = os.path.join(DATA_DIR, 'raw')
CONFIG_DIR = os.path.join(DIR_PATH, 'config')
import yaml
import os

with open(os.path.join(CONFIG_DIR,"master_config.yml"),"r") as f:
    openai_config = yaml.safe_load(f)

with open(os.path.join(CONFIG_DIR, 'input_levels.yml'), 'r') as f:
    input_levels = yaml.safe_load(f)

examples = read_data(join_path(RAW_DIR,'examples.xlsx'))
snacks = pd.read_excel(r"C:\Users\66660\Documents\data_string_parsing.xlsx")

##### User Input File Path

In [26]:
import yaml
import os

with open(os.path.join(CONFIG_DIR,"master_config.yml"),"r") as f:
    openai_config = yaml.safe_load(f)

with open(os.path.join(CONFIG_DIR, 'input_levels.yml'), 'r') as f:
    input_levels = yaml.safe_load(f)

examples = read_data(join_path(RAW_DIR,'examples.xlsx'))
snacks = pd.read_excel(r"C:\Users\66660\Documents\data_string_parsing.xlsx")

c:\Users\66660\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
c:\Users\66660\Anaconda3\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [8]:
input_levels

{'system_content_part1': 'You are a Retail expert with expertise in undertanding SKUs for',
 'super_category': 'snacks',
 'number_of_levels': 5,
 'level1': 'Category of snack',
 'level2': 'Brand Name or Company name',
 'level3': 'Pack size',
 'level4': 'Flavour',
 'level5': 'Type'}

In [27]:
openai.api_key = openai_config['api_key']

#### Functions

In [11]:
def join_path(a,b):
    '''
    Input
    a - folder location
    b - file name

    Return
    joining the path
    '''
    return os.path.join(a,b)

def word_count_from_sentence(text):
    text = text.strip()
    return len(text.split(" "))

def read_data(path):

    try:
        return pd.read_excel(path)
    except:
        return pd.read_csv(path)


def messages_list(examples, input_levels,  choosen_level, input_row,sku_description_column_name):
    messages = []
    system_content = {"role" : "system", "content" : input_levels['system_content_part1'] + " " + input_levels['super_category'] + '.' +  f"In your response, mention ONLY the {input_levels[choosen_level]}."}
    messages.append(system_content)
    for i in range(examples.shape[0]):
        d = {}
        e = {}
        d = {"role" : "user", "content" : f"For this {input_levels['super_category']}, What is the {input_levels[choosen_level]}  of  the following SKU- {examples.user_response.values[i]}?"}
        messages.append(d)
        e = {"role" : "assistant", "content" : f"{examples[choosen_level].values[i]}"}
        messages.append(e)
    messages.append({"role" : "user", "content" : f"For this {input_levels['super_category']}, What is the {input_levels[choosen_level]} of {input_row[sku_description_column_name]}?"})
    return messages


def messages_list_conditional(examples, input_levels,  choosen_level, input_row, sku_description_column_name):
    messages = []
    system_content = {"role" : "system", "content" : input_levels['system_content_part1'] + " " + input_levels['super_category'] + '.' +  f"In your response, mention ONLY the {input_levels[choosen_level]}."}
    messages.append(system_content)
    for i in range(examples.shape[0]):
        d = {}
        e = {}
        d = {"role" : "user", "content" : f"Given the {input_levels['level1']} is {input_row['level1']}. What is the {input_levels[choosen_level]}  of  the following SKU- {examples.user_response.values[i]}?"}
        messages.append(d)
        e = {"role" : "assistant", "content" : f"{examples[choosen_level].values[i]}"}
        messages.append(e)
    messages.append({"role" : "user", "content" : f"What is the {input_levels[choosen_level]} of {input_row[sku_description_column_name]}?"})
    return messages

def retail_gpt(examples,input_levels, choosen_level, input_row, sku_description_column_name,  messages_list, model = 'gpt-3.5-turbo', max_tokens = 50, temperature = 0.6, ):
    response = openai.ChatCompletion.create(
    model = model,        
    messages = messages_list(examples=examples, input_levels=input_levels, choosen_level= choosen_level, input_row = input_row, sku_description_column_name = sku_description_column_name ),
    max_tokens = max_tokens,
    temperature = temperature
    )
    answer = response["choices"][0]['message']['content']
    return answer

def final_prediction(df, examples, input_levels, sku_description_column_name, messages_list,messages_list_conditional):
    df['level1_openai'] = df.swifter.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= 'level1',input_row= x,sku_description_column_name= sku_description_column_name,messages_list= messages_list), axis=1)

    # The for loop will work on levels level2 and onwards, conditional only on level 1
    for i in list(input_levels.keys())[-input_levels['number_of_levels'] + 1:]:
        df[i + '_openai'] = df.swifter.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= i,input_row= x,sku_description_column_name= sku_description_column_name, messages_list= messages_list_conditional), axis=1)

    return df

In [13]:
messages_list_conditional(examples= examples.iloc[0:2], input_levels= input_levels, choosen_level= 'level2', input_row = examples.iloc[0], sku_description_column_name= 'user_response' )

[{'role': 'system',
  'content': 'You are a Retail expert with expertise in undertanding SKUs for snacks.In your response, mention ONLY the Brand Name or Company name.'},
 {'role': 'user',
  'content': 'Given the Category of snack is Nuts. What is the Brand Name or Company name  of  the following SKU- TONG GARDEN PEANUT & CRACKER 50G?'},
 {'role': 'assistant', 'content': 'Tong Garden'},
 {'role': 'user',
  'content': 'Given the Category of snack is Nuts. What is the Brand Name or Company name  of  the following SKU- SALTED SAN TUNG PEANUT 100G?'},
 {'role': 'assistant', 'content': 'San Tung'},
 {'role': 'user',
  'content': 'What is the Brand Name or Company name of TONG GARDEN PEANUT & CRACKER 50G?'}]

In [14]:
retail_gpt(examples= examples.iloc[0:2], messages_list= messages_list_conditional, input_levels= input_levels, choosen_level= 'level5', input_row = examples.iloc[0], sku_description_column_name= 'user_response' )

'peanut and cracker'

In [15]:
data = pd.read_excel(r"C:\Users\66660\Downloads\Everrise_Snacks_Sample_v2.xlsx")
data.head(2)

,Sub-department,Category,Type,Flavour,Brand,Description,Qty Sold,Net Sales,Net Profit,Profit Margin(%),Promo Contri(%),Pack size,Price tier
0,250 - SNACKS,0473 - CORN SNACKS,NaN,NaN,NaN,0821614601 - KOREAN COLOR CURRY POPCORN 80G,305,2020.51,494.84,0.244908,0.123043,Small,Medium
1,250 - SNACKS,0477 - NUTS,Seeds,NaN,Tong Garden,0821620101 - TONG GARDEN SUNFLOWER SEEDS NO SH...,1311,5750.57,1305.02,0.226938,0.071040,Medium,Low


In [16]:
sample = examples.iloc[:20]

sample['level1_openai'] = sample.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= 'level1',input_row= x,sku_description_column_name= 'user_response',messages_list= messages_list), axis=1)

sample.head(2)

C:\Users\66660\AppData\Local\Temp\ipykernel_22748\557422674.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['level1_openai'] = sample.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= 'level1',input_row= x,sku_description_column_name= 'user_response',messages_list= messages_list), axis=1)


,user_response,level1,level2,level3,level4,level5,level1_openai
0,TONG GARDEN PEANUT & CRACKER 50G,Nuts,Tong Garden,small,cracker,peanut,NUTS
1,SALTED SAN TUNG PEANUT 100G,Nuts,San Tung,medium,Salted,peanut,NUTS


In [17]:
list(input_levels.keys())[-input_levels['number_of_levels'] + 1:]

['level2', 'level3', 'level4', 'level5']

In [36]:
snacks.columns

Index(['Sub-department', 'level1', 'level2', 'level3', 'level4', 'Description',
       'Qty Sold', 'Net Sales', 'Net Profit', 'Profit Margin(%)',
       'Promo Contri(%)', 'level5', 'Price tier'],
      dtype='object')

In [29]:
snacks['level1'].unique()

array(['0473 - CORN SNACKS', '0477 - NUTS', '0478 - POTATOES CHIPS',
       '0482 - OTHERS', '0476 - LOCAL SNACK', '0480 - SEAFOOD SNACK',
       '0479 - PRESERVED FRUITS', '0481 - SEAWEED',
       '0471 - CHEESE TWISTY', '0472 - CORN CHIPS', '0475 - LENTIL CHIPS',
       '0474 - FESTIVE', '0470 - CASSAVA CHIPS'], dtype=object)

In [46]:
snacks['word_count'] = snacks.Description.swifter.apply(lambda x : word_count_from_sentence(x))
snacks = snacks.sort_values(by = ['word_count'], ascending = False)
snacks_sub = snacks[snacks.level1.isin(['0477 - NUTS', '0478 - POTATOES CHIPS'])]
snacks_sub.shape

Pandas Apply:   0%|          | 0/1500 [00:00<?, ?it/s]

(512, 14)

In [47]:
snacks_sub.head(5)

,Sub-department,level1,level2,level3,level4,Description,Qty Sold,Net Sales,Net Profit,Profit Margin(%),Promo Contri(%),level5,Price tier,word_count
477,250 - SNACKS,0478 - POTATOES CHIPS,NaN,Salted,Hwa Yuan,2501727201 - HWA YUAN POTATO CHIPS – FLEUR DE ...,74,219.38,-83.42,-0.380253,0.569696,NaN,NaN,16
487,250 - SNACKS,0478 - POTATOES CHIPS,NaN,Salted,Hwa Yuan,2501727001 - HWA YUAN POTATO CHIPS – PINK SALT...,50,200.61,-4.19,-0.020886,0.382384,Small,Medium,13
1126,250 - SNACKS,0478 - POTATOES CHIPS,NaN,Spicy,Hwa Yuan,2501728101 - HWA YUAN POTATO CHIPS – BLACK PEP...,12,47.88,-28.92,-0.604010,1.000000,NaN,NaN,12
679,250 - SNACKS,0478 - POTATOES CHIPS,NaN,Salsa,Mr Potato,2501786501 - MISTER POTATO SYOK WAVE - HOT SAL...,117,643.50,149.70,0.232634,0.000000,Medium,Low,11
29,250 - SNACKS,0478 - POTATOES CHIPS,NaN,Salted,Real Potato,2501769401 - REAL POTATO HANCOOKED SALT & MIL...,12,174.00,31.20,0.179310,0.000000,Medium,Medium,11


In [45]:
snacks.head(2)

,Sub-department,level1,level2,level3,level4,Description,Qty Sold,Net Sales,Net Profit,Profit Margin(%),Promo Contri(%),level5,Price tier,word_count
477,250 - SNACKS,0478 - POTATOES CHIPS,NaN,Salted,Hwa Yuan,2501727201 - HWA YUAN POTATO CHIPS – FLEUR DE ...,74,219.38,-83.42,-0.380253,0.569696,NaN,NaN,16
956,250 - SNACKS,0482 - OTHERS,NaN,NaN,NaN,2391519901 - HERR'S HONEY BBQ CHIP NOW WTH 52 ...,1222,9610.30,1670.03,0.173775,0.635813,Small,Medium,13


In [27]:
snacks.head(2)

,Sub-department,level1,level2,level3,level4,Description,Qty Sold,Net Sales,Net Profit,Profit Margin(%),Promo Contri(%),level5,Price tier
0,250 - SNACKS,0473 - CORN SNACKS,NaN,NaN,NaN,0821614601 - KOREAN COLOR CURRY POPCORN 80G,305,2020.51,494.84,0.244908,0.123043,Small,Medium
1,250 - SNACKS,0477 - NUTS,Seeds,NaN,Tong Garden,0821620101 - TONG GARDEN SUNFLOWER SEEDS NO SH...,1311,5750.57,1305.02,0.226938,0.071040,Medium,Low


In [48]:
examples.shape

(22, 6)

In [53]:
examples.level1.unique()

array(['Nuts', 'LOCAL SNACK', 'OTHERS', 'SEAFOOD SNACK', 'CORN SNACKS',
       'NUTS', 'PRESERVED FRUITS', 'POTATOES CHIPS', 'SEAWEED'],
      dtype=object)

In [54]:
examples = examples[examples.level1.isin(['Nuts','NUTS','POTATOES CHIPS'])]

In [55]:
examples.shape

(9, 6)

In [56]:
output_snacks_sub = final_prediction(df = snacks_sub, examples= examples, input_levels= input_levels, messages_list= messages_list, sku_description_column_name= 'Description', messages_list_conditional= messages_list_conditional)

Pandas Apply:   0%|          | 0/512 [00:00<?, ?it/s]

C:\Users\66660\AppData\Local\Temp\ipykernel_22748\2921351558.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['level1_openai'] = df.swifter.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= 'level1',input_row= x,sku_description_column_name= sku_description_column_name,messages_list= messages_list), axis=1)


Pandas Apply:   0%|          | 0/512 [00:00<?, ?it/s]

C:\Users\66660\AppData\Local\Temp\ipykernel_22748\2921351558.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i + '_openai'] = df.swifter.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= i,input_row= x,sku_description_column_name= sku_description_column_name, messages_list= messages_list_conditional), axis=1)


Pandas Apply:   0%|          | 0/512 [00:00<?, ?it/s]

C:\Users\66660\AppData\Local\Temp\ipykernel_22748\2921351558.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i + '_openai'] = df.swifter.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= i,input_row= x,sku_description_column_name= sku_description_column_name, messages_list= messages_list_conditional), axis=1)


Pandas Apply:   0%|          | 0/512 [00:00<?, ?it/s]

C:\Users\66660\AppData\Local\Temp\ipykernel_22748\2921351558.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i + '_openai'] = df.swifter.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= i,input_row= x,sku_description_column_name= sku_description_column_name, messages_list= messages_list_conditional), axis=1)


Pandas Apply:   0%|          | 0/512 [00:00<?, ?it/s]

C:\Users\66660\AppData\Local\Temp\ipykernel_22748\2921351558.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i + '_openai'] = df.swifter.apply(lambda x : retail_gpt(examples= examples, input_levels= input_levels, choosen_level= i,input_row= x,sku_description_column_name= sku_description_column_name, messages_list= messages_list_conditional), axis=1)


In [57]:
output_snacks_sub.head(2)

,Sub-department,level1,level2,level3,level4,Description,Qty Sold,Net Sales,Net Profit,Profit Margin(%),Promo Contri(%),level5,Price tier,word_count,level1_openai,level2_openai,level3_openai,level4_openai,level5_openai
477,250 - SNACKS,0478 - POTATOES CHIPS,NaN,Salted,Hwa Yuan,2501727201 - HWA YUAN POTATO CHIPS – FLEUR DE ...,74,219.38,-83.42,-0.380253,0.569696,NaN,NaN,16,POTATOES CHIPS,Hwa Yuan,small,Fleur de sel,Potato chips
487,250 - SNACKS,0478 - POTATOES CHIPS,NaN,Salted,Hwa Yuan,2501727001 - HWA YUAN POTATO CHIPS – PINK SALT...,50,200.61,-4.19,-0.020886,0.382384,Small,Medium,13,POTATO CHIPS,Hwa Yuan,small,Pink Salt,Potato chips


In [5]:
output_snacks_sub.to_csv(r"C:\Users\66660\Documents\Projects\bcn-labs-string-parsing\data\outputs\output_snacks_sub.csv")

NameError: name 'output_snacks_sub' is not defined

In [59]:
pip install langchain


                                              0.0/1.4 MB ? eta -:--:--
     --                                       0.1/1.4 MB 2.2 MB/s eta 0:00:01
     ----                                     0.2/1.4 MB 1.9 MB/s eta 0:00:01
     -----                                    0.2/1.4 MB 1.8 MB/s eta 0:00:01
     ---------                                0.3/1.4 MB 2.0 MB/s eta 0:00:01
     -----------                              0.4/1.4 MB 1.9 MB/s eta 0:00:01
     --------------                           0.5/1.4 MB 1.8 MB/s eta 0:00:01
     ---------------                          0.5/1.4 MB 1.7 MB/s eta 0:00:01
     -----------------                        0.6/1.4 MB 1.8 MB/s eta 0:00:01
     --------------------                     0.7/1.4 MB 1.8 MB/s eta 0:00:01
     ---------------------                    0.8/1.4 MB 1.7 MB/s eta 0:00:01
     -----------------------                  0.8/1.4 MB 1.7 MB/s eta 0:00:01
     --------------------------               0.9/1.4 MB 1.7 MB/s eta 

In [60]:
pip install langchain[all]

^C
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain.llms import OpenAI

In [19]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [20]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

In [21]:
messages

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}),
 HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, example=False),
 AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, example=False),
 HumanMessage(content='What is your name?', additional_kwargs={}, example=False)]

In [35]:
OPENAI_API_KEY= openai_config['api_key']

#### Modular Functions

In [39]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9,openai_api_key = openai_config['api_key'])
prompt1 = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt1)

# Run the chain only specifying the input variable.
print(chain.run("colorful socks"))



Vivid Toe-Co.


In [40]:
prompt2 = PromptTemplate(
    input_variables=["company", "product"],
    template="What is a good name for {company} that makes {product}?",
)
chain = LLMChain(llm=llm, prompt=prompt2)
print(chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
    }))



FunkyFeet Socks.


In [42]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9, openai_api_key = openai_config['api_key'])
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("colorful socks"))

Colorful Threads


In [43]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

In [46]:
chain = LLMChain(llm=chat, prompt =  messages)

ValidationError: 1 validation error for LLMChain
prompt
  value is not a valid dict (type=type_error.dict)

In [14]:
levels = {'level1' : 'category' , 'level2' : 'Brand Name of Company name' , 'level3' : 'Pack Size' , 'level4' : 'Flavour',
'level5' : 'Type'}

def system_prompt():
    return input('Enter prompt')

In [18]:
system_prompt = "You are an expert in understanding Retail SKU. You are able to extract information like category, flavour, type and other relevant information from SKU"

In [ ]:
messages = []
system_content = {"role" : "system", "content" : input_levels['system_content_part1'] + " " + input_levels['super_category'] + '.' +  f"In your response, mention ONLY the {input_levels[choosen_level]}."}
messages.append(system_content)
for i in range(examples.shape[0]):
    d = {}
    e = {}
    d = {"role" : "user", "content" : f"For this {input_levels['super_category']}, What is the {input_levels[choosen_level]}  of  the following SKU- {examples.user_response.values[i]}?"}
    messages.append(d)
    e = {"role" : "assistant", "content" : f"{examples[choosen_level].values[i]}"}
    messages.append(e)
messages.append({"role" : "user", "content" : f"For this {input_levels['super_category']}, What is the {input_levels[choosen_level]} of {input_row[sku_description_column_name]}?"})
return messages

In [47]:
def template_level1(system_message = 'You are a Retail expert with expertise in undertanding SKUs. In your response, mention ONLY the category', human_example1 = "What is the category of the following SKU SALTED SAN TUNG PEANUT 100G?" , ai_example1 = f"Nuts",  user_input = 'DAHFA FISH FILLET 120G' ):
    '''
    Prompt template function,
    
    Input:-
    system_message - prompt to design the system
    human_example1 - Giving one example question to the llm
    ai_example - The answer from llm
    user_input - The sku for user input

    Returns:-
    template.format_messages() - Direct input for the llm
    '''
    template = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("human", human_example1),
        ("ai", ai_example1),
        ("human", f"What is the category of {user_input}?"),
    ])
    return template.format_messages()

def output_generation(user_input = 'DAHFA FISH FILLET 120G', system_message = 'You are a Retail expert with expertise in undertanding SKUs. In your response, mention ONLY the category', human_example1 = "What is the category of the following SKU SALTED SAN TUNG PEANUT 100G?" , ai_example1 = f"Nuts"):
    '''This function can be used for output generation for any SKU entry
    Input variables remain the same as template_level1
    '''
    llm = ChatOpenAI(openai_api_key = openai_config['api_key'])
    return llm(template_level1(system_message =  system_message, human_example1 = human_example1, ai_example1= ai_example1, user_input = user_input))

output_generation(user_input = 'DAHFA FISH FILLET 120G')


AIMessage(content='Seafood', additional_kwargs={}, example=False)

In [48]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate

human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template="What is a good name for a company that makes {product}?",
            input_variables=["product"],
        )
    )
chat_prompt_template = ChatPromptTemplate.from_messages([human_message_prompt])
chat = ChatOpenAI(temperature=0.9,openai_api_key = openai_config['api_key'])
chain = LLMChain(llm=chat, prompt=chat_prompt_template)
print(chain.run("colorful socks"))

VibrantSock Co.
ColorSplash Socks
RainbowHue Socks
ChromaSock
SpectrumSock Co.
Kaleidosock
Tie-Dye Toes
Chromatic Threads
ColorfulSoles
Socktastic
Hueful Socks
ColorDrenched Co.
RainbowThreads


In [49]:
def template(system_prompt, human_example = "", ai_example = "", example_on = False):
    a = [
        ("system" , system_prompt + " " + "{level}" ),
    ]
    # if example_on == False:
    #     a.append(("human" , "what is the " + "{level} " + "of " + "{query}" + "?" ))
    #     return ChatPromptTemplate.from_messages(a)
    a.append(("human" , "what is the " + "{level}" + " of the following sku " + human_example ))
    a.append(("ai", ai_example ))
    a.append(("human" , "what is the " + "{level} " + "of " + "{query}" + "?" ))
    return ChatPromptTemplate.from_messages(a)

def messages(system_prompt,  query, user_dictionary, level_number, human_example , ai_example = '',):
    a = template(system_prompt = system_prompt, human_example = human_example, ai_example = ai_example )
    b = a.format_messages(level = user_dictionary[level_number], query = query)
    return b